In [1]:
from transformers import (RobertaConfig,
                          RobertaForSequenceClassification,
                          RobertaTokenizer,
                          AdamW,
                          get_linear_schedule_with_warmup)
import torch
import pytorch_lightning as pl
from torch.optim import lr_scheduler
from torch import nn
import numpy as np

In [2]:
roberta = RobertaForSequenceClassification.from_pretrained('roberta-base')

Some weights of the model checkpoint at roberta-base were not used when initializing RobertaForSequenceClassification: ['lm_head.bias', 'lm_head.dense.weight', 'lm_head.dense.bias', 'lm_head.layer_norm.weight', 'lm_head.layer_norm.bias', 'lm_head.decoder.weight']
- This IS expected if you are initializing RobertaForSequenceClassification from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPretraining model).
- This IS NOT expected if you are initializing RobertaForSequenceClassification from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Some weights of RobertaForSequenceClassification were not initialized from the model checkpoint at roberta-base and are newly initialized: ['classifier.dense.weight', 'classifier.dense.bias', 'classifier.out_proj.weight', 'classifier.out

In [3]:
from Refactored_Baseline import Toy

In [4]:
model = Toy(roberta, lr = .01)

In [5]:
from test_load_data import *

#moral_processor = utils.MoralStoriesProcessor()
print(DatasetType.TRAIN.value)
#the_dataset = MoralStoryDataset("/Users/garbar/Downloads/moral_stories_datasets/classification/action+context/lexical_bias","train")
#training_data = moral_processor.get_train_examples("/Users/garbar/Downloads/moral_stories_datasets/classification/action+context/lexical_bias")
tokenizer = RobertaTokenizer.from_pretrained('roberta-base')
path = "/Users/nissani/Desktop/moral_stories_datasets/classification/action+context/lexical_bias"
datatype = DatasetType.TEST
modeltype = ModelNames.ROBERTA
tasktype = TaskTypes.ACTION_CONTEXT_CLS
the_moral_story = MoralStoryClassifyDataset(path,dataset_type = DatasetType.TEST,tokenizer=tokenizer,model_name=modeltype,tasktype=tasktype)

temp = (the_moral_story._process_data(the_moral_story._moral_story_data[0:10]))
print("OK")

train
OK


In [6]:
the_moral_story.process_data(start_index = 0, stop_index = 28)

In [8]:
some_value = next(iter(the_moral_story))

In [14]:
some_value.input_ids

[3,
 243,
 18,
 1099,
 7,
 28,
 350,
 7337,
 8,
 3874,
 82,
 62,
 4,
 34057,
 13356,
 62,
 182,
 419,
 42,
 662,
 4,
 894,
 1072,
 7,
 146,
 103,
 7080,
 4,
 3,
 3,
 34057,
 11034,
 817,
 1003,
 10,
 5749,
 9,
 25629,
 13,
 7080,
 4,
 3,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0]